In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os, re
import pickle
from utils.imcwrangler import *
%matplotlib inline

%load_ext autoreload
%autoreload 2

DIR = '/home/jerrywang'

In [7]:
DATASET = 'danenbergBreast'
save_dir = f'{DIR}/IMC/output'

if DATASET == 'danenbergBreast':
       df_orig = pd.read_csv(f'{DIR}/geneticbreast/MBTMEIMCPublic/SingleCells.csv')
       markerorder = pd.read_csv(f'{DIR}/geneticbreast/MBTMEIMCPublic/markerStackOrder.csv')
       genename = [gene if gene != 'c-Caspase3c-PARP' else 'c-Caspase3' for gene in markerorder['Epitope'].to_list()]
       df_orig = df_orig[df_orig['is_tumour'] == 1]
       df_orig.rename(columns = {'cellPhenotype':'celltype_rf'}, inplace = True)
       df_orig.loc[df_orig['celltype_rf'] == 'CD8^{+} T cells', 'celltype_rf'] = 'Tcytotoxic'
       tumor_cell = ['Basal', 'CD15^{+}','CK8-18^{+} ER^{hi}', 'CK8-18^{hi}CXCL12^{hi}',
              'CK8-18^{hi}ER^{lo}', 'CK^{+} CXCL12^{+}', 'CK^{lo}ER^{lo}',
              'CK^{lo}ER^{med}', 'CK^{med}ER^{lo}', 'ER^{hi}CXCL12^{+}',
              'Ep CD57^{+}', 'Ep Ki67^{+}', 'HER2^{+}','MHC I & II^{hi}', 'MHC I^{hi}CD57^{+}', 'MHC^{hi}CD15^{+}',]
       # rename all tumor subclasses to tumor cell
       df_orig.loc[df_orig['celltype_rf'].isin(tumor_cell), 'celltype_rf'] = 'Tumor'
       clean_df = df_orig[['ImageNumber','ObjectNumber', 'Location_Center_X', 'Location_Center_Y'] + genename + ['celltype_rf']]
       pat_df = df_orig[['metabric_id','ImageNumber']].drop_duplicates(ignore_index=True)
       pat_df.rename(columns = {'metabric_id':'PatientID'}, inplace = True)
       
       if not os.path.isfile(f'{save_dir}/{DATASET}_patient.csv'):
              pat_df.to_csv(f'{save_dir}/{DATASET}_patient.csv')

elif DATASET == 'cedarsLiver':
       singlecell = pd.read_csv(f'{DIR}/{DATASET}/liver_newdata.csv', index_col=False)
       genename = ['CD45','Glnsynthetase', 'CD163', 'NKG2D', 'CCR4', 'PDL1', 'FAP', 'CD11c',
                   'LAG3', 'HepPar1', 'FOXP3', 'aSMA', 'CD4', 'CD105endoglin', 'CD68',
                   'VISTA', 'CD20', 'CD8a', 'TIM3', 'CXCR4', 'PD1', 'iNOS', 'CD31',
                   'CYR61', 'CDX2', 'CAIX', 'CD3', 'CD44', 'CD15', 'CD11b', 'HLADR',
                   'IL10', 'CXCL12', 'HLAABC', 'DNA1', 'DNA2', 'GranzymeB', 'Ki67',
                   'HistoneH3', 'CXCR3', 'Galectin9', 'YAP', 'CD14', 'CK19']
       column_mapping = {'sample':'PatientID', 'broad_type':'celltype_rf',}
       clean_df = singlecell.rename(columns=column_mapping)
       clean_df.loc[clean_df['liver_pos'], 'celltype_rf'] = 'Liver'
       clean_df.loc[clean_df['tumor_pos'], 'celltype_rf'] = 'Tumor'
       clean_df.loc[clean_df['Mac_pos'], 'celltype_rf'] = 'Macrophage'
       clean_df.loc[clean_df['Tcell_pos'], 'celltype_rf'] = 'Tcell'
       clean_df.loc[clean_df['CD8_pos'], 'celltype_rf'] = 'Tcytotoxic'
       clean_df.loc[clean_df['M1_pos'], 'celltype_rf'] = 'M1'
       clean_df.loc[clean_df['NK_pos'], 'celltype_rf'] = 'NK cells'
       clean_df.loc[clean_df['Treg_pos'], 'celltype_rf'] = 'Tregs'
       clean_df.loc[clean_df['MDSC_pos'], 'celltype_rf'] = 'MDSC'
       clean_df.loc[clean_df['DC_pos'], 'celltype_rf'] = 'DC'
       clean_df.loc[clean_df['B_pos'], 'celltype_rf'] = 'B cells'
       clean_df.loc[clean_df['M2_pos'], 'celltype_rf'] = 'M2'
       clean_df.loc[clean_df['CD4_pos'], 'celltype_rf'] = 'CD4'
       
       clean_df = clean_df[['ImageNumber',] + list(column_mapping.values()) + 
                           ['Location_Center_X', 'Location_Center_Y'] + genename ]
       pat_df = clean_df[['PatientID','ImageNumber']].drop_duplicates(ignore_index=True)
       if not os.path.isfile(f'{save_dir}/{DATASET}_patient.csv'):
              pat_df.to_csv(f'{save_dir}/{DATASET}_patient.csv')

elif DATASET == 'crcSchurch':
       # groups = 1 is CLR, groups = 2 is DII
       # Get cell masks stored in tiff files
       datapath = f'{DIR}/{DATASET}/CRC_clusters_neighborhoods_markers.csv'
       singlecell = pd.read_csv(datapath)
       column_mapping = {'ROI_ID':'ImageNumber', 'Region':'ImageNumber',  'patients':'PatientID', 
                         'X:X':'Location_Center_X','Y:Y':'Location_Center_Y','ClusterName':'celltype_rf',}
       
       # remove MMP9 and CD30 from the analysis (don't actually have any intensity in raw image)
       channelnames = ['CD44 - stroma:Cyc_2_ch_2', 'FOXP3 - regulatory T cells:Cyc_2_ch_3',
              'CD8 - cytotoxic T cells:Cyc_3_ch_2',
              'p53 - tumor suppressor:Cyc_3_ch_3',
              'GATA3 - Th2 helper T cells:Cyc_3_ch_4',
              'CD45 - hematopoietic cells:Cyc_4_ch_2', 'T-bet - Th1 cells:Cyc_4_ch_3',
              'beta-catenin - Wnt signaling:Cyc_4_ch_4', 'HLA-DR - MHC-II:Cyc_5_ch_2',
              'PD-L1 - checkpoint:Cyc_5_ch_3', 'Ki67 - proliferation:Cyc_5_ch_4',
              'CD45RA - naive T cells:Cyc_6_ch_2', 'CD4 - T helper cells:Cyc_6_ch_3',
              'CD21 - DCs:Cyc_6_ch_4', 'MUC-1 - epithelia:Cyc_7_ch_2','CD2 - T cells:Cyc_7_ch_4',
              'Vimentin - cytoplasm:Cyc_8_ch_2', 'CD20 - B cells:Cyc_8_ch_3',
              'LAG-3 - checkpoint:Cyc_8_ch_4', 'Na-K-ATPase - membranes:Cyc_9_ch_2',
              'CD5 - T cells:Cyc_9_ch_3', 'IDO-1 - metabolism:Cyc_9_ch_4',
              'Cytokeratin - epithelia:Cyc_10_ch_2',
              'CD11b - macrophages:Cyc_10_ch_3', 'CD56 - NK cells:Cyc_10_ch_4',
              'aSMA - smooth muscle:Cyc_11_ch_2', 'BCL-2 - apoptosis:Cyc_11_ch_3',
              'CD25 - IL-2 Ra:Cyc_11_ch_4', 'CD11c - DCs:Cyc_12_ch_3',
              'PD-1 - checkpoint:Cyc_12_ch_4',
              'Granzyme B - cytotoxicity:Cyc_13_ch_2', 'EGFR - signaling:Cyc_13_ch_3',
              'VISTA - costimulator:Cyc_13_ch_4', 'CD15 - granulocytes:Cyc_14_ch_2',
              'ICOS - costimulator:Cyc_14_ch_4',
              'Synaptophysin - neuroendocrine:Cyc_15_ch_3',
              'GFAP - nerves:Cyc_16_ch_2', 'CD7 - T cells:Cyc_16_ch_3',
              'CD3 - T cells:Cyc_16_ch_4',
              'Chromogranin A - neuroendocrine:Cyc_17_ch_2',
              'CD163 - macrophages:Cyc_17_ch_3', 'CD45RO - memory cells:Cyc_18_ch_3',
              'CD68 - macrophages:Cyc_18_ch_4', 'CD31 - vasculature:Cyc_19_ch_3',
              'Podoplanin - lymphatics:Cyc_19_ch_4', 'CD34 - vasculature:Cyc_20_ch_3',
              'CD38 - multifunctional:Cyc_20_ch_4',
              'CD138 - plasma cells:Cyc_21_ch_3', 'HOECHST1:Cyc_1_ch_1',
              'CDX2 - intestinal epithelia:Cyc_2_ch_4',
              'Collagen IV - bas. memb.:Cyc_12_ch_2',
              'CD194 - CCR4 chemokine R:Cyc_14_ch_3',
              'MMP9 - matrix metalloproteinase:Cyc_15_ch_2',
              'CD71 - transferrin R:Cyc_15_ch_4', 'CD57 - NK cells:Cyc_17_ch_4', 'DRAQ5:Cyc_23_ch_4']
       gene_mapping = {channel:channel.split(":")[0].split(" - ")[0] for channel in channelnames}
       genename = list(gene_mapping.values())
       column_mapping.update(gene_mapping)
       clean_df = singlecell.rename(columns=column_mapping)
       clean_df = clean_df[list(column_mapping.values())]
       clean_df.loc[clean_df['celltype_rf']=='tumor cells', 'celltype_rf'] = 'Tumor'
       clean_df.loc[clean_df['celltype_rf']=='CD8+ T cells', 'celltype_rf'] = 'Tcytotoxic'
       # converting region number to unique image numbers based on TMA_12 == 1 or 2
       mapping_dict = {label:int((re.search(r"reg(\d+)", label)).group(1)) for label in np.unique(clean_df['ImageNumber'])}
       clean_df['ImageNumber'] = clean_df['ImageNumber'].map(mapping_dict)
       clean_df['ImageNumber'] = clean_df['ImageNumber'] + (singlecell['TMA_12']-1)*np.max(clean_df['ImageNumber'])
       # converting XY position units from pixel (0.377um/pixel) to micron
       clean_df['Location_Center_X'] = clean_df['Location_Center_X']*0.377
       clean_df['Location_Center_Y'] = clean_df['Location_Center_Y']*0.377

       pat_df = clean_df[['PatientID','ImageNumber']].drop_duplicates(ignore_index=True)
       if not os.path.isfile(f'{save_dir}/{DATASET}_patient.csv'):
              pat_df.to_csv(f'{save_dir}/{DATASET}_patient.csv')

elif DATASET == 'hochMelanoma':
       df_orig = pd.read_csv(DIR + '/data_for_analysis_zenodo/rna/cell.csv')

       # Keeping only necessary columns
       columns_to_keep = ['ImageNumber','ObjectNumber', 'Location_Center_X', 'Location_Center_Y']
       intensity_columns = [names for names in df_orig.columns if "MeanIntensityCorrected_FullStackFiltered" in names]
       columns_to_keep = columns_to_keep + intensity_columns
       RNA_df = df_orig[columns_to_keep]

       # Label channels using gene name instead of numbers
       genename = ["Vimentin","CD163","B2M","CD134","CD68","GLUT1","CD3","Lag3","PD1","CCL4_mRNA","CCL18_mRNA",
              "HistoneH3","CCR2","PDL1","CXCL8_mRNA","CXCL10_mRNA","CXCL12_mRNA","CXCL13_mRNA","CD8","CCL2_mRNA",
              "CCL22_mRNA","CXCL9_mRNA","SMA","DapB_mRNA","SOX10","CCL8_mRNA","CD31","CCL19_mRNA","Mart1","pRB",
              "cleavedPARP","DNA1","DNA2","CK5","CD15","MPO","CD38","HLADR","S100","Cadherin11","FAP"]
       RNA_df = RNA_df.rename(columns=dict(zip(intensity_columns,genename)))

       # Adding celltype information from a different csv file
       columns_to_add = ['celltype','celltype_rf']
       RNA_celltype = pd.read_csv(DIR + '/data_for_analysis_zenodo/rna/celltype_RNA.csv')
       clean_df = pd.concat([RNA_df, RNA_celltype[columns_to_add]],axis=1)
       # load image metadata as output from CellProfiler
       image_mat = pd.read_csv(DIR + "/data_for_analysis_zenodo/rna/Image.csv")
       scaling = image_mat['Scaling_FullStack'][1]
       print(scaling)

: 

In [ ]:
print(clean_df['Location_Center_X'].min())
print(clean_df['Location_Center_Y'].min())

1.0

In [ ]:
# sub = clean_df[['ImageNumber','Location_Center_X', 'Location_Center_Y','celltype_rf']]
# sub = sub.rename(columns={'Location_Center_X': 'x', 'Location_Center_Y': 'y', 'celltype_rf': 'label'})
# sub.to_csv('danenbergBreast_cells.csv', index=False)

In [ ]:
from sklearn.preprocessing import StandardScaler
df = clean_df[genename]
# create a StandardScaler object
scaler = StandardScaler()
# fit and transform the data
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

fano = clean_df[list(genename)].std()/clean_df[list(genename)].mean()
plt.figure(figsize=(20,8))
plt.bar(range(len(fano)), fano)
plt.xticks(range(len(fano)),fano.index, rotation=80, fontsize=12)
plt.axhline(y=1, color='r')
plt.show()

### Gene expression by cell type

In [ ]:
unwanted_num = []
x = clean_df.sort_values(by=['celltype_rf'])[[ele for ele in genename if ele not in unwanted_num]]
normalized_x = x.div(x.sum(axis=1), axis=0)
normalized_x['celltype_rf'] = clean_df['celltype_rf']
normalized_x = normalized_x.sort_values(by=['celltype_rf']).groupby(['celltype_rf']).mean()

import seaborn as sns
import matplotlib.colors as colors

ax=sns.clustermap(normalized_x,z_score=1,
                  row_cluster=False,
                  cmap="vlag", vmin=-3, vmax=3, figsize=(20,10),
                  yticklabels=normalized_x.index, xticklabels=normalized_x.columns)
plt.setp(ax.ax_heatmap.get_xticklabels(), rotation=80)
plt.rc('xtick') 
plt.ylabel('Normalized levels')
plt.show()

### Generating dataset

In [ ]:
clean_df

In [ ]:
patch_sz = 48
pixel_dim = 3
width = int(patch_sz/pixel_dim)
height = int(patch_sz/pixel_dim)
df = image_to_patch(clean_df, (patch_sz,patch_sz))
df2 = patch_to_pixel(df, width=width, height=height, pixel_dim=(pixel_dim,pixel_dim))
df2[['x0','y0']] = df[['x0','y0']]

In [ ]:
tcount_per_patch = df2.groupby('ImageNumber').apply(lambda x: (x['celltype_rf'] == 'Tcytotoxic').sum()).reset_index(name='counts')

In [ ]:
unique, counts = np.unique(tcount_per_patch['counts'].values, return_counts=True)

In [ ]:
# Print the unique values and their counts
for u, c in zip(unique, counts/np.sum(counts)*100):
    print(f"{u}: {c}")

In [ ]:
# remove marker channels
channel_to_remove=[]
genes_to_keep = [i for i in genename if i not in channel_to_remove]

In [ ]:
# further remove all signals from T cells
celltype = 'Tcytotoxic'
df2.loc[df2['celltype_rf']==celltype, genes_to_keep] = 0

# extract coordinate of each patch
position = get_patch_coord(df2, patch_sz)

In [ ]:
np.save('position.npy', position)

In [ ]:
intensity, label, genes_to_keep, groupedimage = patch_to_matrix(df2, width=width, height=height, 
                                                                typeName='celltype_rf', 
                                                                genelist=genename, 
                                                                celltype=['Tcytotoxic','Tumor'], channel_to_remove=channel_to_remove)
intensity = np.swapaxes(np.swapaxes(intensity, 1, 2), 2, 3)
label = label.astype(int)

# Get back original image number label
original_ImageNumber = np.array(groupedimage[['ImageNumber','original_ImageNumber']].drop_duplicates()\
                                ['original_ImageNumber'])
orig_imagenum = groupedimage[['ImageNumber','original_ImageNumber']].drop_duplicates()
orig_imagenum = orig_imagenum.set_index('ImageNumber')
label = pd.concat([orig_imagenum, label], axis=1).reset_index()
label['ImageNumber'] = label['original_ImageNumber'].astype(int)
label = label.drop(columns=['original_ImageNumber'])

In [ ]:
label.shape

In [ ]:
test = np.max(intensity,axis=3)
n_cell_per_patch = np.sum(test>0, axis=(1,2))
plt.hist(n_cell_per_patch)
plt.xlabel('number of cells per patch')
print(f"median number of cells per patch:{np.median(n_cell_per_patch)}")

In [ ]:
nrow = 10
ncol = 10
samp = np.random.randint(0,np.size(test,axis=0),size=(nrow,ncol))

fig, axs = plt.subplots(nrows=nrow, ncols=ncol, figsize=(ncol*2,nrow*2))

for ii in range(nrow):
    for jj in range(ncol):
        for ind in samp:
            axs[ii][jj].imshow(test[samp[ii,jj],:,:])

In [ ]:
# check percent of cells merged as a result of discretization
print('Percent of cells lost: {:.2f}'.format((len(df2)-np.sum(np.max(intensity,axis=-1)>0))/len(df2) * 100))

In [ ]:
# Save data
data_dir = f'{DATASET}_sz{patch_sz}_pxl{pixel_dim}_nc{len(genes_to_keep)}'
if not os.path.exists(f'{save_dir}/{data_dir}'):
    os.makedirs(f'{save_dir}/{data_dir}')
with open(f'{save_dir}/{data_dir}/patched.dat', 'wb') as f:
    pickle.dump((intensity, label, genes_to_keep, position), f, protocol=4)
if not os.path.isfile(f'{save_dir}/{data_dir}/singlecell_df.csv'):
    # note this df doesn't contain cd8 t cells
    df2.to_csv(f'{save_dir}/{data_dir}/singlecell_df.csv')

Plotting cell distribution

In [ ]:
# subset_df = clean_df[clean_df['celltype_rf'].isin(['Tumor', 'Tcytotoxic'])].copy(deep=True)

In [ ]:
%matplotlib inline

subset_df = clean_df.copy()
ncelltype_rf = len(subset_df['celltype_rf'].unique())
cmap = cm.get_cmap('Set1')
# color_dict = {'Tumor':'tab:red','Tcytotoxic':'tab:green'}
# Passing a number betwen 0-1 into cmap will return a color to me
color_dict = {value:cmap(count/ncelltype_rf) for count, value in enumerate(subset_df['celltype_rf'].unique())}
color_series = pd.Series(color_dict)

# Naming my series so I can merge it below. Only named series can be merged
color_series.name = 'color_series'

# # Merge your two datasets
color_df = pd.merge(subset_df[['ImageNumber','Location_Center_X','Location_Center_Y','celltype_rf']], 
                  color_series.to_frame(), how='left', left_on='celltype_rf', right_index=True)

plt.figure(figsize=(12, 4))
plt.scatter(range(ncelltype_rf), np.zeros(ncelltype_rf), 
            c=np.array([color_dict[key] for key in color_dict]),s=100)
for i in range(ncelltype_rf):
    plt.text(i, 0.005, list(color_dict)[i], rotation=45, fontsize=12)
plt.axis('off')
plt.show()

high_cell_images = list((color_df['ImageNumber'].value_counts()>500).index[color_df['ImageNumber'].value_counts()>500])

plt.figure(figsize=(30, 80))
plt.suptitle("Breast tumor slices", fontsize=25, y=0.9)
for n, ticker in enumerate(high_cell_images[40:50]):
    plt.subplot(10, 4, n+1)
    singleimage = color_df.loc[color_df['ImageNumber'] == ticker]
    plt.scatter(singleimage['Location_Center_X'],singleimage['Location_Center_Y'],4,c=singleimage['color_series'])
plt.savefig('cell_map.png', dpi=300)

In [ ]:
# data for causal discovery
df3 = df2[['original_ImageNumber','ImageNumber','celltype_rf']]
df3 = pd.get_dummies(df3, columns=['celltype_rf'], prefix='', prefix_sep='')
df3 = df3.groupby('ImageNumber').max().reset_index(drop=True)
df3.rename(columns={'original_ImageNumber':'ImageNumber'}, inplace=True)
df3["PatientID"] = df3["ImageNumber"].map(dict(zip(pat_df.ImageNumber, pat_df.PatientID)))
df3.to_csv('/home/jerrywang/IMC/output/danenbergBreast_cellcount.csv', index=False)